<a href="https://colab.research.google.com/github/rzl-ds/gu511/blob/master/015_hadoop.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# `hadoop`

## `emr` spinup

we will want to work with an `emr` `hadoop` cluster throughout this lecture, but it takes about 12 or so minutes to fully spin up. Normally, I would say: Just start it!

But as this class is async, quick note of caution.

**this cluster will cost 0.745 USD**

if you just want it on during your watching of the lecture, go for it. However, if you want to pause, watch part today and part tomorrow, etc -- you may want to wait to start your cluster until you need it.

and on the other end: **be sure you shut off your cluster when you are done, or if you aren't going to use it for several hours**

**<div align="center">starting up an `emr` cluster</div>**

the below steps will create a cluster which will cost about 0.74 USD per hour.

+ in the `aws` web console open the `emr` service
+ click create cluster, and on the "quick options" screen select "advanced options"
+ software and steps
    + stay at emr-5.31.0
    + for software, click:
        + `hadoop`
        + `ganglia`
        + `hive`
        + `hue`
        + `spark`
        + `livy`
        + `pig`
    + notice but don't click: `jupyterhub`, `mxnet`, `tensorflow`
    + click next
+ hardware config -- leave all defaults
    + generally, think about space requirements [ala this](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-plan-instances-guidelines.html#emr-plan-instances-hdfs)
+ general options -- pick a name
+ security options -- ***choose a key pair! make sure you have that key!!***

## the problem(s)

reach all the way back in your memory several lectures ago, when we talked about `nosql` databases, and in particular document stores like `aws` `dynamodb`. the proposed use case for these `nosql` databases was an ambiguous "webby" one:

*we're reading and writing way too much data way too fast for our one machine*

this sentiment is reflective of a modern data reality that often goes by the buzziest of buzzwords:

**big data**

*mandatory caveat: big data $\neq$ data science*

traditional data analyses were optimized for super-powerful single machines such as the monolithic, super-powerful `sql` servers

*note: this is not to say that cluster computing didn't exist; indeed it was one of the main computational frameworks in the early days of computers*

our exponential growth in disk space and memory space per dollar spent fueled a lot of this work and innovation.

basically, for a long while our ability to *compute* data grew faster than our ability to *create* or *acquire* data. in the modern world, though, that notion is absolute history.

take, for example, a relatively trivial process for modern computation: word counts for a document of several MBs:

In [ ]:
%%bash
rm /tmp/shakespeare.txt 2> /dev/null
wget --quiet -O /tmp/shakespeare.txt.zip https://github.com/bbengfort/hadoop-fundamentals/raw/master/data/shakespeare.txt.zip
unzip /tmp/shakespeare.txt.zip -d /tmp
ls -alh /tmp/shak*

In [ ]:
%%bash
head /tmp/shakespeare.txt

In [ ]:
with open('/tmp/shakespeare.txt', 'r') as f:
    s = f.read()

print(s[:100])

In [ ]:
import collections

wordct = collections.Counter(
    word.lower()
    for line in s.split('\n')
    for word in line.strip().split('\t')
    if word)

wordct.most_common(10)

*don't mind me, just cleaning up...*

In [ ]:
!rm /tmp/shakespeare.txt*

that was easy, but it relied on some important features:

1. I had enough disk space to have that 8.5MB file stored locally
2. I had enough memory to load that 8.5MB file's contents directly into memory

obviously, that isn't always the case. It's not even hard to think of counter-examples

1. a larger text corpus (e.g. all of wikipedia, 10TB as of 2015, or publicly available SEC EDGAR filings)
2. any reasonably large image recognition project
3. the logs of web traffic for any modestly sized website or service
4. IoT information (usage records of your smartphone or headphones, e.g.)

so, back to `dynamodb`. when we (theoretically) started to run into resource issues for our single-machine architecture, we decided to change the way we were doing things and start "scaling horizontally" -- choose an architecture and software that can spread the storage and computation burden across multiple machines

for `dynamodb` we were attempting to distribute out our database writes and reads as actions, but the test scenario I laid out above was one of

+ data storage
+ resource availability

`hadoop` is a distributed storage and computing software, and although it's dominance is waning (especially as `spark` takes over many of its use cases), it is still the primary big data distributed storage and ETL processing

it is a software solution that abstracts out all the "hard stuff" (the complicated networking and resource marshaling) that needs to happen to get multiple computers on the same page, and instead provides the user (you) with a single api for

+ accessing distributed files (`hdfs`)
+ securing computational resources and memory (`yarn`)

## `hadoop` nuts and bolts

let's dig into the details of distributed computing a bit

### terminology

+ **node**: a single machine (real or virtual)
+ **cluster**: a collection of *nodes* which can communicate with each other
+ **master**: a *node* which can request information from or delegate tasks to other *nodes*
+ **worker**: a *node* which merely receives, processes, and responds to requests

+ **local**

up until now, we have cultivated a healthy habit of calling our laptop "local" and our `ec2` instance "remote", but we will break that temporarily here to talk about `hdfs`.

in the `hadoop` context, when we say "local" we generally mean whatever machine we are on when we are executing `hadoop` commands. note that this can be our (up to now exclusively "remote") `ec2` server.

### basic concepts

in the database world we had certain requirements a `dbms` needed to meet to ensure that all clients of that database service could share those resources (the `ACID` principles)

similarly, for distributed computing to be well defined and robust, we have four requirements:

1. *fault tolerance*: if one computer goes down, we're good. if it comes back, we're even gooder
2. *recoverability*: we don't lose data when things fail
3. *consistency*: results shouldn't depend on jobs failing or succeeding
4. *scalability*: more data means longer time, not failure; we can increase resources if desired

`hadoop` addresses these requirements by making the following decisions:

+ data is distributed across many nodes in the cluster; each node prefers its local data
+ all data is chunked into blocks (say, 128 MB) and is *replicated* (copied to other nodes)

<img src="https://d1jnx9ba8s6j9r.cloudfront.net/blog/wp-content/uploads/2016/10/Replication-Management-Apache-Hadoop-HDFS-Architecture-Edureka-Blog.png" width="800px">

+ jobs (computations) are broken into tasks applied to single blocks
+ jobs are completely unaware that they are distributed
+ worker nodes don't care about each other
+ tasks are redundant and repeatable
+ master nodes handle allocation of all resources (storage, cpus, memory)

### `hadoop` architecture

`hadoop` is open source software which defines a number of utilities, but the two most important are:

1. `hdfs` (a program for handling distributed file storage)
2. `yarn` (a program for handling distributed resource allocation)

together, these two services work together to enforce some of those design decisions above: namely, to make sure that all data is robustly distributed and that all distributed tasks are working on local data.

`hdfs` handles the distribution of files (e.g. how is data chunked into blocks? where did I leave block 1337? who do I ask to send that block to me?)

`yarn` handles the distribution of tasks, aka requests for compute resources (e.g. I was asked to calculate a word count, how do I break this task up in a way my lovable but dumb worker nodes will understand?)

`hdfs` and `yarn` are the default software services for managing distributed storage or compute resources - and they were specifically designed to work in tandem - but either can be replaced or augmented:

+ you could change storage methods (e.g. `hdfs` replaced by `s3` or `dbfs`)
+ you could change resource managers or computational layers on top of storage (e.g. `yarn` replaced by `spark` or `mesos`)

#### a `hadoop` cluster

`hadoop` is a software. the hardware is a cluster of computers. the benefit you get in using `hdfs` and `yarn` are abstracted `api`s that hide cluster administration details and tasks from you.

to put it another way: `hadoop` lets someone else do the hard task of distribution so you can do what you came here to do (data processing).

the cost of deferring that hard work is that you must fit your analysis into that abstracted `api` -- not always easy!

in this class, we've seen many instances of the client / service paradigm (e.g. when making `ssh` connections, when making `REST` requests, when connecting to databases)

both `hdfs` and `yarn` have several *services* that our tools (*clients*) use

`hdfs` services:

+ `NameNode` (master): stores the directory tree, file metadata, file cluster locations. this is the access point for `hdfs` usage
+ `Secondary NameNode` (master): performs housekeeping, checkpointing. this is more like an assistant `NameNode` than a backup `NameNode`
+ `DataNode` (worker): local `io` for `hdfs` blocks

the basic interaction with `hdfs`:

1. client asks `NameNode` where data lives.
2. `NameNode` tells client
3. client is responsible for going and getting data from `DataNode`

`yarn` services:

+ `ResourceManager` (master): allocates and monitor resources (memory, cores), schedules jobs
+ `ApplicationMaster` (master): coordinates a particular app after `ResourceManager` has scheduled it
+ `NodeManager` (worker): runs tasks and reports on task status

the basic interaction with `yarn` is very similar:

1. client asks `ResourceManager` for resources
2. `ResourceManager` assigns `ApplicationMaster` instance to manage the individual application
3. `ApplicationMaster` submits a job to a single `NodeManager`, tracks all submitted jobs
4. `NodeManager` executes incoming assigned tasks

to give you a sense of scale for a typical `hadoop` cluster

+ 20 - 30 workers and one master can handle 10s of terabytes of data in simultaneous workflows
+ to get into hundreds of nodes at a time, you will need to start migrating master services to separate servers (no more single master nodes)
+ to get into the thousands of nodes, you will need to create multiple master service machines just for communication between the nodes

#### details on `hdfs`

`hdfs` is a file system on top of another filesystem. in many respects, it behaves like you're used to the `linux` filesystem behaving (with slightly different commands). there are a few nuances worth discussing, however.

##### blocks

files are blocked into large (e.g. 128MB) chunks. this means that a file larger than that will be chopped up into different blocks.

for example, if I have a 450 MB file `corpus.txt` and a block size of 128 MB per block, `hadoop` will break up that `corpus.txt` file into chunks and distribute them (perhaps with replication) to different workers. schematically (the names don't exist in real life):

+ `corpus.txt.1` (0 - 128 MB)
+ `corpus.txt.2` (128 - 256 MB)
+ `corpus.txt.3` (256 - 384 MB)
+ `corpus.txt.4` (384 - 450 MB)

it's worth noting: unlike block sizes in non-distributed file systems, in `hadoop` a small file will not wastefully take up the remainder of the space on the hard drive.

that's not to say there isn't a problem with small files, though -- there is. It's not wasteful disk usage, it's wasteful *resource* usage. we will discuss `mappers` and `reducers` later, but for now it suffices to say: when we distributed tasks, we already said we distribute them to each needed block.

if we have many small files we will have to keep track of many different blocks, and every time we perform a task we'll have to create many sub-tasks to do so. that will be pretty wasteful.

in the end: better a million files of 100 MB than a billion files of 0.1 MB

**<div align="center">PAUSE FOR ZOOM BREAK</div>**

## `aws` `emr`: amazon's `hadoop`

so you have decided you have enough data you need to move to a distributed environment -- you could buy several servers and install `hadoop` on them (a great exercise for masochists!)... or you could pay `aws` to do that for you. that's obviously what we've done.

we stressed earlier that `hadoop` is a software, not a hardware.

one implementation of that software (really, a modification) is the `aws` version of `hadoop`, which is called `e`lastic `m`ap `r`educe, or `emr`.

at the start of the class we created a `hadoop` cluster using the `aws` `emr` service -- it should be done by now, so let's go back and re-visit what we did.

we asked amazon to create a 3-node cluster with

+ 1 master node (running the `NameNode` `hdfs` services and the `Resource-` and `ApplicationManager` `yarn` services)
+ 2 **core** nodes (running the `DataNode` `hdfs` service and the `NodeManager` `yarn` service)
    + in `aws`, *core* nodes have `hdfs` blocks. all tasks using `hdfs` will rely on the *core* nodes
+ 0 **task** nodes
    + in `aws`, *task* nodes **do not** have `hdfs` blocks on them, and are instead available largely for computation purposes

additionally, being an `aws` product, there is immediate integration with other `aws` services, and in particular `s3`.

`emr` can use `hdfs` (in `aws` land, that is ephemeral storage that is destroyed when a cluster is taken down), or it can use `s3`, or it can use the local file systems on the `aws` `ec2` servers which are actually running the cluster.

generally, input and output is done in `s3` and intermediate steps are retained in the ephemeral `hdfs` storage

we also asked amazon to install several common `hadoop` technologies. we will explain them as we use them throughout the lecture

### pricing

`emr` pricing is not immediately obvious, but is simple enough. you pay a certain fee for the `emr` service depending on the underlying machine type you choose (e.g. `m5.xlarge`), and you *also* pay for those instances themselves (as if they were just `ec2` nodes you started), and you *also* pay for the attached `ebs` disk space (it's not a bundled deal!).

for our demo we put together above, we made a 3-node cluster with `m5.xlarge` instances. as of writing, the rates for this `instance` type are:

+ compute costs
    + 0.192 USD per hour for each instance
    + 0.048 USD per hour for the `emr` service for each instance
    + three instances means `3 * (0.192 + 0.048) = 0.72`, 0.72 USD per hour
+ storage costs
    + 0.10 USD per GB-month
        + given that we have 192 GB total (64 GB EBS for each machine by default), this is about 0.0258 USD per hour

all total we're seeing 0.745 USD per hour. 3 hours is around 2.24 USD

we're burning money!

let's use our clusters already. log in to the `master` node as you would any other `ec2` instance: via `ssh`

**<div align="center">logging in to your `emr` cluster</div>**

+ in the `aws` web console open the `emr` service and select your `emr` cluster
+ update the security group of your master node to have your (or all) public ip addresses
    + click on the security group linked under "security groups for master"
    + there should be two records on this page -- click on the `master` group
    + add an *inbound* rule allowing `ssh` type traffic from all ip address
+ on the "summary" tab, find the value of "Master public DNS"
    + e.g. mine was `ec2-34-231-180-183.compute-1.amazonaws.com`
+ using the key pair you specified when you created the cluster, execute (replacing the `####` characters with your dns value we just grabbed)
    + `ssh -i /path/to/your/keypair.pem hadoop@ec2-###-###-###-###.compute-1.amazonaws.com`
    + note user is `hadoop` here, not `ubuntu` or `ec2-user`

## working with a distributed file system

### basic file system operations

many of the common `linux` command line file system tools are available with the same names in `hadoop`. try

```bash
hadoop fs -help
```

(note the single-dash parameters and curse the `java` gods)

tired of reading those 4000 lines? try any one subcommand too:

```bash
hadoop fs -help ls
hadoop fs -help chmod
```

let's prepare our `hadoop` cluster to actually do some `hadoop`-y stuff. on the `emr` master node, run:

```bash
# get the corpus locally
mkdir ~/code && cd ~/code
sudo yum install -y git
git clone https://github.com/bbengfort/hadoop-fundamentals.git
cd hadoop-fundamentals/data
unzip shakespeare.txt.zip

# put it into hdfs
hadoop fs -mkdir /data
hadoop fs -put shakespeare.txt /data/shakespeare.txt
hadoop fs -ls -h /data
```

the opposite of `put`-ting a file is `get`-ting a file, in which we can `get` items from `hdfs` and download them to our local hard drive

```sh
hadoop fs -get /data/shakespeare.txt /tmp/test_get.txt
ls -alh /tmp/test*
rm /tmp/test_get.txt
```

the files we save in `hadoop` are generally enormous. it's good to know right away how to read portions of such large files:

+ `hadoop fs -cat /data/shakespeare.txt | less`
+ `hadoop fs -cat /data/shakespeare.txt | head` (this aborts the streaming when `head` has had enough)
+ `hadoop fs -tail /data/shakespeare.txt`

#### quick refresher on `uri`s and `schema`s

above we executed our first `hdfs` file system operation -- a `put`. the `put` command will take a source file (assumed to be *local*, on your regular `emr` master node's hard drive) and will `put` it into a destination path in `hdfs`.

we are talking about a path on the *local* file system and then a path in `hdfs` and using the same syntax -- just `/`-separated paths. we are relying on the `put` command *knowing* which is which

generally speaking, we may need to be more explicit than this. recall from the database class that we can uniquely describe any resource (`REST` api endpoints, files, etc) with a `url` or `uri` that includes a schema, a la

```
schema://[user[:password]@]host[:port]/path
```

secretly, `hadoop` has been converting all the arguments we've been giving it into `uri`s

for both the local (`ec2`) file system and `hdfs`, the `user`, `password`, and `port` don't have to be provided (this is always the case for the local `ec2` file system, and handled for `hdfs` by the way that `hadoop` has been configured for us by `aws`). furthermore, `hadoop` uses the `HOSTNAME` environment variable for the host value if none is provided. this collapses our `uri`s to

```
schema:///path
```

for files we want to access, the schemas are

+ local file system: `file`
+ `hdfs`: `hdfs`

therefore, to describe the local file, the full path would be

```
file:///home/hadoop/code/hadoop-fundamentals/data/shakespeare.txt
```

and the `hdfs` full path would be

```
hdfs:///data/shakespeare.txt
```

to verify that the `HOSTNAME` value is used by the `hadoop` command, try:

```sh
hadoop fs -tail hdfs://$HOSTNAME/data/shakespeare.txt
```

#### a tangent: `hdfs dfs` vs. `hadoop fs`

floating around out on the etherwebs, you may see stack overflow posts with commands such as

```bash
hdfs dfs -ls /
```

note that we're not writing that here, but instead writing `hadoop fs` instead of `hdfs dfs`

`hdfs dfs` is *related to* `hadoop fs`, but is not exactly the same. `hadoop fs` defaults to looking at `hdfs` files, but is actually file-system agnostic(ish), and supports local files (via the `file://` schema), `s3` files, `ftp` services, and any other schema people have been kind enough to implement.

`hdfs dfs` *only* works with `hdfs`.

to demonstrate how `hadoop fs` can be used with local files as well, try out

```bash
hadoop fs -ls file:///tmp/
```

and compare the results to

```bash
ls -alh /tmp/
```

none of this is to say *exclusively prefer* `hadoop fs` or *avoid* `hdfs dfs`. just knowing what the difference is may help you avoid some confusion when you try the subcommands or flags of one and don't experience the same result as you would with the other.

## web interfaces to `hadoop` services

many of the `hadoop` services available via command line actions executed on our `ec2` instances are also available via web interfaces. as a matter of security, `aws` will only allow them to be accessed locally from that node. our best option is to use **`ssh` port forwarding** --  effectively, we are just replacing a port on our local machine with a single port on the remote machine.

```bash
ssh -i /path/to/your/keypair.pam -NfL $LOCAL_PORT:$REMOTE_PORT hadoop@ec2-###-###-###-###.compute-1.amazonaws.com
```

**<div align="center">set up `ssh` port forwarding</div>**

*windows users: check out [this walkthrough](https://www.akadia.com/services/ssh_putty.html)*

+ you are running these commands on *your laptop*, not the `ec2` instance
+ verify you can make an `ssh` connection
    + you did this in the "logging in to your emr cluster" exercise above
+ set the values of `MASTER_DNS` and `KEY_PAIR` below in a `bash` session **on your local laptop**

```sh
MASTER_DNS=YOUR_MASTER_DNS  # e.g.: ec2-3-229-137-205.compute-1.amazonaws.com
KEY_PAIR=YOUR_KEY_PAIR      # e.g.: ~/.ssh/my_special_key.pem
LOCAL_YARN=8088
LOCAL_HDFS=50070
LOCAL_SPARK=18080
LOCAL_HUE=8887

# YARN ResourceManager
ssh -i $KEY_PAIR -NfL $LOCAL_YARN:$MASTER_DNS:8088 hadoop@$MASTER_DNS

# Hadoop HDFS NameNode
ssh -i $KEY_PAIR -NfL $LOCAL_HDFS:$MASTER_DNS:50070 hadoop@$MASTER_DNS

# Spark HistoryServer
ssh -i $KEY_PAIR -NfL $LOCAL_SPARK:$MASTER_DNS:18080 hadoop@$MASTER_DNS

# Hue
ssh -i $KEY_PAIR -NfL $LOCAL_HUE:$MASTER_DNS:8888 hadoop@$MASTER_DNS
```

after the above we have

+ `YARN ResourceManager`: [http://localhost:8088](http://localhost:8088)
+ `Hadoop HDFS NameNode`: [http://localhost:50070](http://localhost:50070)
+ `Spark HistoryServer`: [http://localhost:18080](http://localhost:18080)
+ `Hue`: [http://localhost:8887](http://localhost:8887)
    + note: I used local port 8887 to avoid my `jupyter` server (ip is reserved), but you don't have to

**<div align="center">optional: set up `ssh` port forwarding for non-master-node services</div>**

+ go to emr web console > cluster page > summary tab
    + click on the security group link for "Security groups for Core & Task"
    + make sure your current IP address has `ssh` access to it (security group settings, people!)
+ go to emr web console > cluster page > hardware tab
    + scroll in the table to the two public dns names
    + verify `ssh` works with a simple `ssh -i /path/to/key.pem hadoop@hadoop@ec2-###-##-##-###.compute-1.amazonaws.com`
+ replace the `CORE_DNS_1`, `CORE_DNS_2`, and `KEY_PAIR` values below with values of your choosing and run this **on your local laptop**

```sh
CORE_DNS_1=YOUR_CORE_DNS_1
CORE_DNS_2=YOUR_CORE_DNS_2
KEY_PAIR=YOUR_KEY_PAIR
LOCAL_YARN_1=8042
LOCAL_YARN_2=8043
LOCAL_HDFS_1=50075
LOCAL_HDFS_2=50076

# YARN NodeManager
ssh -i $KEY_PAIR -NfL $LOCAL_YARN_1:$CORE_DNS_1:8042 hadoop@$CORE_DNS_1
ssh -i $KEY_PAIR -NfL $LOCAL_YARN_2:$CORE_DNS_2:8042 hadoop@$CORE_DNS_2

# Hadoop HDFS DataNode
ssh -i $KEY_PAIR -NfL $LOCAL_HDFS_1:$CORE_DNS_1:50075 hadoop@$CORE_DNS_1
ssh -i $KEY_PAIR -NfL $LOCAL_HDFS_2:$CORE_DNS_2:50075 hadoop@$CORE_DNS_2
```

after the above we have

+ `YARN NodeManager` for core 1: [http://localhost:8042](http://localhost:8042)
+ `YARN NodeManager` for core 2: [http://localhost:8043](http://localhost:8043)
+ `Hadoop HDFS DataNode` for core 1: [http://localhost:50075](http://localhost:50075)
+ `Hadoop HDFS DataNode` for core 2: [http://localhost:50076](http://localhost:50076)

### the `hdfs` `NameNode` webapp

for starters, we have a web interface for exploring the `hdfs` directory structure: http://localhost:50070/explorer.html#/. there isn't much to see at this point, but this is one alternative interface to the `hadoop` `cli`

for example, go here: http://localhost:50070/explorer.html#/data to see the `shakespeare.txt` file we `put` into `hdfs` earlier this lecture

### `hue`

the `h`adoop `u`ser `e`xperience -- or `hue` for short -- is an extremely useful web interface. this is a sort of one-stop-shop for interacting with several of the most prominent `hadoop` tools -- especially for defining and organizing jobs.

after the port forwarding steps above, `hue` should be accessible at http://localhost:8887 or http://localhost:8888, depending on your choice of `LOCAL_HUE` port

**<div align="center">`hue`-based `emr`-walkthrough</div>**

+ first, create an admin login. ***USE USERNAME `hadoop`***. use whatever password you want, but please use username `hadoop`!!
+ do the simple guided walkthrough `hue` provides
+ talk about the different utilities together

**<div align="center">exercise: actually using `hue` to interact with `hadoop`</div>**

let's put some data out there. go to the `hue` file browser

+ textcorpus
    + on your laptop: `wget https://github.com/bbengfort/hadoop-fundamentals/blob/master/data/textcorpus.zip?raw=true -O textcorpus.zip`
    + in `hue`, click on the "Files" icon on the left panel menu
    + click the "upload" button and upload `textcorpus.zip`
    + click the uploaded item and click the "Extract" button
+ shakespeare
    + on your laptop: `wget https://github.com/bbengfort/hadoop-fundamentals/blob/master/data/shakespeare.txt.zip?raw=true -O shakespeare.txt.zip`
    + repeat the above
+ python data
    + on your laptop: `wget https://s3.amazonaws.com/hadoop.rzl.gu511.com/pyq_stripped.tar.gz`
    + repeat (note: this may take a while!)
+ note: you may get an error or two during extract, ignore

returning now to the **editor** (the `</>` icon in the top left) -- this app provides a single interface to edit different queries in `hadoop`-relevant languages:

+ `sql`-like query commands: `hive`, `sparksql`, generic databases (`mysql`, `postgresql`, etc)
+ `hadoop`-specific languages and tools: `pig`, `mapreduce`, `sqoop1`
+ `spark` commands: `scala`, `spark`, and `pyspark`

let's try using it to run a simple `pyspark` snippet against our `pyq_stripped` dataset. don't worry about not knowing `pyspark` just yet -- we just want to demonstrate it's possible

```python
import pyspark.sql.functions as F

answers_file_name = 'hdfs:///user/hadoop/pyq_stripped/Answers.csv'

z = spark.read.csv(answers_file_name, header=True)

z.agg(F.avg('Score')).show()
```

**<div align="center">PAUSE FOR ZOOM BREAK</div>**

## working with distributed computing

as we said above, `yarn` is the main resource manager and one of the main access points for computation. in the original instance of `hadoop`, however, the computational framework was a software called `mapreduce`

knowing what `mapreduce` is helps illuminate the engineering paradigm at play in `hadoop` programs

### `mapreduce`: a functional programming model

`mapreduce` was [first proposed](https://research.google.com/archive/mapreduce.html) by google developers as a way of performing easily distributable computations

the name comes from the two "pieces":

+ a `map` function takes input as a series of key-value pairs ("kvps") and performs the same computation on each pair, generating a (possibly empty) sequence of intermediate kvps
    + this is often where analysis happens (usually)
    + e.g. filter: take a key, check if it belongs in a list of acceptable keys, emit the kvp if yes, pass silently if no
+ a `reduce` function takes a key and an iterator of values and process the values, usually to determine some aggregate statistic

these functions ought to be stateless functional programming functions

take this diagram representing a simple map-reduce pair which, end-to-end, make a word count job

+ the `map` step takes a single input row (`A B R`) and emits a list of kvps which are the elements of the row and the number `1` (`A, 1`, `B, 1`, `R, 1`)
+ the `reduce` step takes a list of `?, 1` values and *reduces* it by counting the `1`s (summing them)

<img src="https://www.tutorialspoint.com/map_reduce/images/mapreduce_work.jpg" width="700px">

#### `mapreduce`: implemented on a cluster

the `mapreduce` framework is great for a distributed computation environment because it is assumes many of the central tenets of the distribution framework. specifically, because mappers and reducers are stateless functions, they can be executed by a worker node to independently work on any number of blocks and emit their responses back to the master node.

mappers are already set: individual blocks are key-value pairs where the keys are file or line metadata and the values are the contents of the file / line. we can distribute the mapper function to any number of workers and let them process blocks at their own pace without any outside information

did the machine `map`-ping `A B R` die? just ask the other machine with that file block to process it!

reducers needs all the output values for a single key across all processed blocks, so we have to wait until all mappers are done to "reduce".

we create as many reducers as there are output keys and distribute them among the workers

because reducers expect to get the keys emitted by mappers and **all** values for those keys, we need to perform a shuffle and sort of those intermediate kvps before we can reduce. this stage is called exactly that: *shuffle and sort*

so, in the end, we have a general framework:

+ input: `hdfs` kvps
+ mapping: input kvps are processed by mappers and generate intermediate kvps
+ shuffle and sort: take the generated key, partition the key space, and assign keys to reducers
+ reduce: take the keys and the iterated list of values and reduce them to aggregate kvps

it's kvps all the way down!

##### mapreduce examples

we already counted words in the shakespeare corpus, in memory in plain `python`, and the pseudo-code which can fit this wordcount problem into `mapreduce` is not that different:

```python
def mapper(documentkey, line):
    for word in line.split():
        emit(word, 1)

def reducer(word, values):
    emit(word, sum(val for val in values))
```

#### submitting a mapreduce job to `yarn`

`yarn` is responsible for scheduling tasks, so if we would like to perform some task we need to give it to `yarn`.

one way (and the most basic) is to create a `jar` file (compiled `java` code) and to pass that directly to `yarn` using the `hadoop jar` command.

in [the github repo](https://github.com/bbengfort/hadoop-fundamentals) for the "Data Analytics with Hadoop" O'Reilly book, we have been provided with a couple `java` files to implement a simple `mapreduce` word count job

+ [`WordCount.java`](https://github.com/bbengfort/hadoop-fundamentals/blob/master/wordcount/WordCount/WordCount.java)
+ [`WordMapper.java`](https://github.com/bbengfort/hadoop-fundamentals/blob/master/wordcount/WordCount/WordMapper.java)
+ [`SumReducer.java`](https://github.com/bbengfort/hadoop-fundamentals/blob/master/wordcount/WordCount/SumReducer.java)

let's compile and run that code on the shakespeare corpus.

first thing's first, let's compile our `java` code into a `jar` file (run this on your master node)

```bash
export HADOOP_CLASSPATH=$JAVA_HOME/java/lib/tools.jar
cd ~/code/hadoop-fundamentals/wordcount/WordCount/
javac *.java -cp $(hadoop classpath)
jar cf wc.jar WordCount.class WordMapper.class SumReducer.class
```

final thing's final, we can submit the `jar` file to `yarn` by calling

```bash
# use hadoop to submit a jar
# that jar is wc.jar
# the class to invoke is WordCount
# the input is a file /data/shakespeare.txt
# the output files go in a direcotry named wordcounts
hadoop jar wc.jar WordCount /data/shakespeare.txt wordcounts
```

we can track the results of that job via a web interface at http://localhost:8088

the output is now saved in `hdfs`:

```sh
hadoop fs -ls wordcounts/
hadoop fs -cat wordcounts/part-r-00000 | head -n20
```

**<div align="center">PAUSE FOR ZOOM BREAK</div>**

### not using `java`  with `hadoop` streaming

so we were able to take someone else's writen `java` code to create `mapreduce` jobs. super.

I mean... not knowing `java` is a bit of a problem though. not to be ungrateful.

this is just what we get out of the box with `hadoop` `mapreduce`.

+ `java` `api` with input, output, map and reduce functions, job params exposed as *job configuration*
+ jobs get packaged into a `jar` file which is passed to the `ResourceManager` by the *job client*
+ `ResourceManager` handles the rest

but what if you don't want to write `java` code that implements this same workflow over and over and over again?

or just don't want to write `java` code *at all*, because you already did everything you needed to do in `python`?

*hadoop streaming* is here to help.

### `hadoop` streaming

hadoop streaming is a specific `java` util which can take any executable (in *any* language!) and use that as a mapper or reducer or combiner.

really, this is just a super hacky `jar` file that is submitted in the same was as our `wc.jar` in our example above. for this `hadoop`-specific `jar` file, you pass executable scripts or commands as parameters to this `jar` file

note: the word "streaming" is used because the input and output method is unix streams (`stdin`, `stdout`), not in reference to streaming data.

this is actually pretty cool, because we know how to access those streams:

+ `python`: `sys` module
+ `R`: `file("stdin")` (I think? who even knows. does anyone?)

when we develop a `mapper.py` script, know the following:

+ *each* mapper launches the executable. long spin-up times suck for obvious reasons
+ `hadoop streaming` parses input data into lines of text and pipes them through `stdin`
+ `python` streaming script parses those lines of texts and prints (to `stdout`) kvps delimited in some way (default is `\t`)
+ these intermediate kvps are scooped up by `hadoop streaming` again and passed on to the reducer
+ the mapper gets an entire block via `sys.stdin`. so it doesn't receive a *file*, or a *line number*, it receives a file handler to a block. that's important.

the `reducer.py` script follows much of the same logic, but in addition:

+ the reducer doesn't receive a key and an iterable, it reads shuffled and sorted kvp records (like a table) from stdin (they are in the `a\tb` format)
+ a single reducer task will always get *all* records for given key, but *may* get more than one key (so your reducer doesn't have a key, we need logic there)

for both files (and for any file in any language being used as a `hadoop streaming` script), the shebang (`#!`) declaration at the top of the file is important -- it tells the streaming process (a bash shell) how to execute the script (e.g. in `python`)

#### real example: flight data

the [bureau of transportation statistics](https://transtats.bts.gov/) makes [on-time flight data](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time) publicly available.

let's download some and use a pre-written `mapper.py` and `reducer.py` to calculate the average delay per airport

first, on your `emr` master node, download the airline data:

```bash
cd /tmp
wget --no-check-certificate -O flights.zip https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_1.zip
unzip flights.zip
# annoying file name!
mv On_Time_Reporting_Carrier_On_Time_Performance_\(1987_present\)_2020_1.csv flights.csv
```

the code we will use is in

```bash
cd ~/code/hadoop-fundamentals/avgdelay
```

let's take a look at `mapper.py` and `reducer.py` in that repo.

one nice thing about this simple framework is that we can test our functions in a simple series of pipes:

```bash
head -n100 /tmp/flights.csv | ./mapper.py | sort | ./reducer.py
```

so, we have seen that the `avgdelay` code is able to `map` and `reduce` the records in the `csv` of airport delays we downloaded. let's ship that over to `hdfs` and run a `streaming` job with these files

```bash
hadoop fs -put /tmp/flights.csv .
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper.py,reducer.py \
    -input flights.csv \
    -output average_delay \
    -mapper mapper.py \
    -reducer reducer.py

# trust, but verify
hadoop fs -cat average_delay/part* | head -n25
```

note the `-files` params -- what's going on there?

in a cluster environment, the materials of the executed code will generally have to be either

1. pre-installed on the cluster, so that it is obvious
2. shipped along with the request to execute.

**<div align="center">PAUSE FOR ZOOM BREAK</div>**

## the limitations of `hadoop`

so now that we've spent a bunch of time on `hadoop`, it's time to come clean:

`hadoop` is pretty hard to use (do *you* want to write `mapper.py` and `reducer.py` jobs for everything you do?), and is becoming less useful every day as other technologies - especially distributed database and `spark` -- gain popularity and efficiency.

particularly for iterative data science workflows, `pyspark` is *much* closer to the ideal workflow. `hadoop` is very specifically geared for `etl` processes -- files in, files out. if you want to do repeat iteration on objects in an interactive environment, `hadoop` is very much not for you!

the important thing to know about `hadoop`, in the end, is that it has some use cases for which it is very legitimately **the** solution to the problem (maybe only, but definitely standard). that being said, it is often **a pretty bad** solution to a problem, but one that the people who came before you suggested for... reasons

if you are thinking of using `hadoop` (or being told to use `hadoop`), ask yourself

+ do I have a fixed input and a fixed output?
+ is this a single-shot process that I want to repeat?
+ am I done with exploration?

if so, `hadoop` may still be a good choice. if not, you probably want `spark` -- or something else entirely

## software lightning round

### `hadoop` and `spark`

the current leaders in the distributed data and distributed computation sphere.

`hadoop` is the on-disk distributed file processing tool which we've talked about here

`spark` is the in-memory analog of `hadoop` and we will discuss this at length in the following lecture

### `pig`

[`pig`](https://pig.apache.org/) is a declarative querying / scripting language (not like `sql`, more like `spark`). basically this is a higher-order abstraction of `mapreduce` functions that can be extremely fast while also being much easier to read and write

### `hive`

[`hive`](https://hive.apache.org/) is the `sql` of the `hadoop` ecosystem. it also doubles as a data warehousing platform for many of the other applications. it implements batch querying (not interactive querying), and is not *itself* a database, just a means of interacting with other data structures

I personally find that `hive` is the easiest of the `hadoop` ecosystem tools to spin up in (especially if you have a background in `sql`), so you may want to consider staring here.

### `hcatalog`
[`hcatalog`](https://cwiki.apache.org/confluence/display/Hive/HCatalog) is a sub-component of `hive` that sees wide use in other applications. it is the table and storage management layer, and is used by disparate apps to provide a normalized, gridded view of data of many different formats. it acts as a normalized layer between application and data format

### `oozie`

[`oozie`](http://oozie.apache.org/) is the scheduling and workflow management tool for `hadoop` jobs. it uses `xml` configuration files to define chained DAGs of jobs, executes them, reports on results and logs, handles concurrency and timing issues, and provides for a complex set of control flows.

personally I think it's pretty janky, but its' the best you've got.

### `hue`

[`hue`](http://gethue.com/) (short for `h`adoop `u`ser `e`xperience) aims to be the way that users experience `hadoop`, and that's a great thing -- almost everything you might want to do in the `hadoop` ecosystem will be easier (at first) to do in `hue`.

over time, of course, complications and bugs will push you down (up?) into the command line and the programs themselves, but for beginning it is *essential* that you try using `hue`.

**<div align="center">quick `hive` example using `hue`</div>**

+ open `hue` at `http://localhost:8887` or `http://localhost:8888`
    + earlier we should have uploaded and extracted a set of `python` questions from stack overflow
+ create new database `stack_overflow_python`
    + click the table icon on the left > click the `+` icon > follow the wizard for each of the three unzipped files
    + note: creating a table **removes** the `csv` from the original `hdfs` location
+ do some simple queries

### `mahout`

[`mahout`](http://mahout.apache.org/) is a framework for building machine learning applications. it is not immediately clear to me the extent to which `mahout` and `spark`'s `mllib` are *competitors* or *complimentors*, and I haven't been able to clear that up.

### `sqoop`

[`sqoop`](http://sqoop.apache.org/) (`sq`l `o`n hado`op`) is a bulk data transfer took, looking to implement (under the hood) `mapreduce` jobs to perform data transfer between traditional databases, `hdfs`, and other data sources.

### `tez`
[`tez`](https://tez.apache.org/) is similar to `spark` in that it is positioning itself as a replacement for `mapreduce` for simpler application frameworks. however, it is trying to be much closer to the `hadoop` / `yarn` infrastructure, implementing a normalized `api` so that other tools (the `pig`s and `hive`s of the world) can run directly off of `tez` in a normalized way.

### `hbase`
[`hbase`](https://hbase.apache.org/) is a big data database and date store. it is actually a `nosql` key-value store on `hdfs`, and it has its own query language (like many `nosql` dbs)

### `phoenix`
[`phoenix`](https://phoenix.apache.org/)

an on-line transaction processing software for performing data transactions in a `sql` syntax, but using `hbase` as its backing (so schemaless).

### `presto`
[`presto`](https://prestodb.io/) is a `sql` query engine for interactive queries against big data platforms. it was developed at facebook so it has good integration with `cassandra` in addition to other data stores.

it's main goal is to be interactive and very fast. I haven't used it myself but the reviews are pretty positive.

### `zeppelin`
[`zeppelin`](https://zeppelin.apache.org/) is the `jupyter notebook` of the `spark` world, allowing for collaborative and exploratory work to be done in a web-based notebook. current interpreters include `spark`, `sql`, `python`, `hive`, `pig`, `sparksql`, `markdown`, and others

### `flink`
[`flink`](https://flink.apache.org) is a stream processing framework. many of the big data sources of import are streams of data (log files, e.g.) and `flink` aims to be the standard means of persisting data streams into distributed environments

### `zookeeper`
[`zookeeper`](https://zookeeper.apache.org/) is a centralized configuration engine. with so many moving parts in the distributed cluster environment, this can be pretty essential

### `livy`
[`livy`](https://livy.incubator.apache.org/) is a `REST` interface for `spark`, allowing users to submit `spark` jobs from anywhere (not just on the cluster, in a `repl`)

### `ganglia`
[`ganglia`](http://ganglia.sourceforge.net/) is the primary distributed monitoring system for `hadoop` clusters and grid computing

### `mxnet`
[`mxnet`](https://mxnet.incubator.apache.org/) is an apache incubator project for developing distributed deep learning algorithms

## DELETE YOUR `emr` CLUSTER!

don't forget to do it! if you are going to work on a homework assignment feel free to leave it up, but if you aren't *immediately* doing that, consider deleting it and cloning it when you are ready for your hw assignment

<strong><em><div align="center">`hadoop` is `hadope`</div></em></strong>
<div align="center"><img src="https://pbs.twimg.com/media/Bnz0UglCUAA_Kb-.jpg"></div>

# END OF LECTURE

## appendix

the following are some extra items I thought would be useful to share but are not really a part of this lecture

### a development `hadoop` environment

a single `hadoop` (or, related, `aws` `emr`) environment is often a large, complicated, expensive, and unruly engineering project.

to avoid the hassle of constantly building up complicated development environments, many developers will create a *virtual execution environment* in either a *virtual machine* or a *`docker` container*

#### using `virtualbox` and a pre-built vm image

we are going to build one such virtual environment right now using oracle's `virtualbox` and the Ubuntu 14.04 `vmdk` provided by the authors of our text book

**<div align="center">walkthrough: installing `virtualbox` and a `hadoop` virtual machine</div>**

1. download https://resources.oreilly.com/examples/0636920035275/raw/master/hfpd3.vmdk.gz
2. download `virtualbox` for your os and follow instructions: https://www.virtualbox.org/wiki/Downloads
4. unzip the `vmdk` file once it is downloaded
    1. in a terminal, `gunzip -k hfpd3.vmdk.gz`
5. create the VM
    1. open `virtualbox` and click the "new" button
    2. name it whatever you want, change the type to `linux`, and make the version "ubuntu 64-bit"
    3. set the memory however you want (I'll go high because yolo)
    4. select "Use an existing virtual hard disk file" and navigate to the `vmdk` file
    5. start up the VM. password is `password`
    6. click "Devices > Insert guest additions cd" and the run (again, password is `password`)
    7. restart the VM when finished (now you can resize!)
    8. back in the `virtualbox` program, navigate to "Settings", and on the "General > Advanced" tab make "Shared Clipboard" "Bidirectional"

**<div align="center">starting `hadoop`</div>**

1. log in to your `hadoop` vm
2. execute the following:

```bash
sudo -H -u hadoop $HADOOP_HOME/sbin/start-dfs.sh
sudo -H -u hadoop $HADOOP_HOME/sbin/start-yarn.sh
sudo -H -u hadoop $HADOOP_HOME/bin/hadoop fs -chown -R hadoop:hadoop /
sudo -H -u hadoop $HADOOP_HOME/bin/hadoop fs -chown -R student:student /user/student
sudo -H -u hadoop $HADOOP_HOME/bin/hadoop fs -chmod g+w /
sudo chmod g+w /var/app/hadoop/data

# demonstrate it worked
hadoop fs -mkdir -p /user/student
hadoop fs -ls -h /
```

#### using a `cloduera` `docker` `image`

there are a few publicly available `docker` `image`s distributed by `cloudera`. several older versions exist on `dockerhub` and have an incredibly low barrier to entry as a result of this.

the more up-to-date versions can be found on [the `cloudera` quickstart page](https://www.cloudera.com/downloads/quickstart_vms/5-13.html), and though they are also easy to install, it requires some identifying information to sign up and I try to limit that when possible.

both images are pretty excellent and well documented so I highly recommend them for your general development and hacking.

for both of the `docker` images below, you should increase your vm memory as discussed [here for mac / windows](https://stackoverflow.com/questions/44533319/how-to-assign-more-memory-to-docker-container). I recommend `8 GB`

for linux, just increase memory by invoking `docker run` (when you do) with the command line flag `--memory=8g`

##### cloudera 5.7

this version is freely available on `dockerhub` and accessible via

```sh
docker pull cloudera/quickstart:latest
```

##### cloudera 5.13

back on [the `cloudera` quickstart page](https://www.cloudera.com/downloads/quickstart_vms/5-13.html) you have the option to sign up and download a cloudera quickstart `docker` `image` for the most recent version of the cloudera distribution. if you choose to do this, installation instructions can be found [here]().

for me, on my mac, the following are sufficient:

```sh
tar -xzvf cloudera-quickstart-vm-5.13.0-0-beta-docker.tar.gz
cd cloudera-quickstart-vm-5.13.0-0-beta-docker
docker import cloudera-quickstart-vm-5.13.0-0-beta-docker.tar
```

this takes a couple of minutes even if it's working as planned. when it finishes it will print out a `sha` (for example, mine was `sha256:e7701e8fac26ab951ee2b404594c43cac7d5e8f00b6b1d1dac73f1d6ede48904`). copy that value and add a tag:

```sh
docker tag e7701e8fac26ab951ee2b404594c43cac7d5e8f00b6b1d1dac73f1d6ede48904 cloudera/quickstart:5.13
docker tag e7701e8fac26ab951ee2b404594c43cac7d5e8f00b6b1d1dac73f1d6ede48904 cloudera/quickstart:latest
```

##### running your `cloudera` `docker` `image`

whichever `image` you acquired above, you can run it with

```sh
docker run \
    --hostname=quickstart.cloudera \
    --privileged=true \
    -rm \
    -it \
    -p $HOST_PORT_HUE:8888 \
    -p $HOST_PORT_CLOUDERA_MANAGER:7180 \
    -p $HOST_PORT_TUTORIAL:80 \
    $CLOUDERA_QUICKSTART_IMAGE_NAME
    /usr/bin/docker-quickstart
```

for the `$CLOUDERA_QUICKSTART_IMAGE_NAME` value, you are looking for the name of the `image` you just created. you can find this from `docker images`

for the 5.7 `image`, this is certainly `cloudera/quickstart:latest`. for the 5.13 `image`, this is whatever tag values you entered above (e.g. `cloudera/quickstart:5.13

for example, the following oneliner has my preferred ports (I didn't want to use 8888 (`jupyter` was running) or `80` (I have a web server running):

```sh
docker run --hostname=quickstart.cloudera --privileged=true --rm -it -p 9999:8888 -p 7180:7180 -p 8880:80 cloudera/quickstart:latest /usr/bin/docker-quickstart
```

in the 5.7 `image`, `hue` *appears* to fail to start this way, but in my experience it succeeds (on a delay of a few seconds). if not, instructions on how to get that to work are [here](https://medium.com/@SnazzyHam/how-to-get-up-and-running-with-clouderas-quickstart-docker-container-732c04ed0280), but basically you will just run

```sh
service hue start
```

inside the `container` and check again.

for what it's worth, a `cloudera` proprietary tool called cloudera manager isn't started by default. ***it recommends the VM have at least 8 GB memory to run this (we set 8 above)***. if you want to run (not recommended at this time), execute

```sh
/home/cloudera/cloudera-manager --express --force
```

in addition to the command line prompt the above `docker run` command will drop you at, this will create several links to which you may now navigate:

+ [hue](http://localhost:9999)
    + username and password: `cloudera`
+ [cloudera manager](http://localhost:7180)
+ [tutorial](http://localhost:8880)

#### `cloudera` version comparison

| observable     | 5.6            | 5.13            |
|----------------|----------------|-----------------|
| java version   | 1.7.0_67       | 1.7.0_67        |
| hadoop version | 2.6.0-cdh5.7.0 | 2.6.0-cdh5.13.0 |
| spark version  | 1.6.0          | 1.6.0           |
| hive version   | x.y.z          | 1.1.0           |
| hue version    | 3.9.0          | 3.9.0           |

useful links

+ [quora article on distribution and rack awareness](https://www.quora.com/How-does-HDFS-split-files)
+ [great `hdfs` tutorial](https://www.edureka.co/blog/apache-hadoop-hdfs-architecture/?utm_source=quora&utm_medium=crosspost&utm_campaign=social-media-edureka-ab)
+ [seemingly up-to-date table of `hadoop` ecosystem components](https://hadoopecosystemtable.github.io/)
+ [deeper dive on small file size impacts](https://community.hitachivantara.com/community/products-and-solutions/pentaho/blog/2017/12/15/working-with-small-files-in-hadoop-part-1)